In [49]:
import gpxpy
import pandas as pd
import numpy as np
import math
import pdb
import folium
#from folium.plugins import MiniMap
from folium import plugins
from IPython.display import display, HTML
import branca
from collections import namedtuple
import xml.etree.ElementTree as ET
import os
%matplotlib inline

In [50]:
def process_gpx_to_df(file_name):

    gpx = gpxpy.parse(open(file_name))  
    
    #(1)make DataFrame
    track = gpx.tracks[0]
    segment = track.segments[0]
    # Load the data into a Pandas dataframe (by way of a list)
    data = []
    segment_length = segment.length_3d()
    for point_idx, point in enumerate(segment.points):
        data.append([point.longitude, point.latitude,point.elevation,
                     point.time, segment.get_speed(point_idx)])
    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
    gpx_df = pd.DataFrame(data, columns=columns)
    
    #2(make points tuple for line)
    points = []
    for track in gpx.tracks:
        for segment in track.segments:        
            for point in segment.points:
                points.append(tuple([point.latitude, point.longitude]))
    
    return gpx_df, points

In [51]:
df, points = process_gpx_to_df("Tolentino_Urbisaglia.gpx")

In [53]:
mymap = folium.Map( location=[ df.Latitude.mean(), df.Longitude.mean() ], zoom_start=12, tiles=None)
folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}', attr="Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA, ESA, METI, NRCAN, GEBCO, NOAA, iPC", name='Nat Geo Map').add_to(mymap)
folium.TileLayer('http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg', attr="terrain-bcg", name='Terrain Map').add_to(mymap)
folium.PolyLine(points, color='red', weight=4.5, opacity=.5).add_to(mymap)

#get start and end lat/long
lat_start = df.iloc[0].Latitude
long_start = df.iloc[0].Longitude
lat_end = df.iloc[-1].Latitude
long_end = df.iloc[-1].Longitude

#build starting marker
html_camino_start = """Start of day""".format()
popup = folium.Popup(html_camino_start, max_width=400)
#nice green circle
folium.vector_layers.CircleMarker(location=[lat_start, long_start], radius=9, color='white', weight=1, fill_color='green', fill_opacity=1,  popup=html_camino_start).add_to(mymap) 
#OVERLAY triangle
folium.RegularPolygonMarker(location=[lat_start, long_start], 
      fill_color='white', fill_opacity=1, color='white', number_of_sides=3, 
      radius=3, rotation=0, popup=html_camino_start).add_to(mymap)


html_camino_name = """
<div align="center">
<h5>Daily Stats</h5><br>
</div>

"""

#get midpoint long / lad
length = df.shape[0]
mid_index= math.ceil(length / 2)

lat = df.iloc[mid_index]['Latitude']
long = df.iloc[mid_index]['Longitude']

html = html_camino_name
popup = folium.Popup(html, max_width=300)
 
folium.Marker([lat, long], popup=popup, icon=folium.Icon(color='red', icon_color='white', icon='bicycle', prefix='fa')).add_to(mymap)

#END MARKER
html_camino_end = """End of day""".format()
popup = html_camino_end
folium.vector_layers.CircleMarker(location=[lat_end, long_end], radius=9, color='white', weight=1, fill_color='red', fill_opacity=1,  popup=popup).add_to(mymap) 
#OVERLAY square
folium.RegularPolygonMarker(location=[lat_end, long_end], 
      fill_color='white', fill_opacity=1, color='white', number_of_sides=4, 
      radius=3, rotation=45, popup=popup).add_to(mymap)

folium.LayerControl(collapsed=True).add_to(mymap)


mymap.save("map_name.html")# saves to html file for display below
mymap